# Get a Constellation graph
Get some vertex and transaction attributes.

In [ ]:
import json
from PIL import Image
import io

import constellation_client

In [ ]:
SPRITE_ATLAS = 'sprite_atlas.png'

# Constellation icons are this big.
#
ICON_SIZE = 256

# We build a texture map this big.
# Even low-end GPUs should be able to cope with this.
#
TEXTURE_SIZE = 4096

TEXTURE_ICONS = TEXTURE_SIZE // ICON_SIZE
print(TEXTURE_ICONS)

In [ ]:
cc = constellation_client.Constellation()

## Vertices

In [ ]:
vx_attrs = ['source.[id]', 'source.Label', 'source.x', 'source.y', 'source.z', 'source.icon', 'source.background_icon', 'source.color', 'source.nradius']

vxdf = cc.get_dataframe(vx=True, attrs=vx_attrs)
vxdf = vxdf.rename(columns=lambda name:name.replace('source.', '', 1) if name.startswith('source.') else name)
vxdf.head()

## Icons

In [ ]:
def make_texture_atlas(icons, atlas_name):
    """Given a collection of icons of size ICON_SIZE, create a texture map.
    
    TODO put the background icons in first?
    
    :returns: a map from name to cell_index
    """
    
    icons = set(icons)
    
    N = len(icons)
    W = min(N, TEXTURE_ICONS)
    H = N % W + 1
    if H >= TEXTURE_ICONS:
        raise ValueError('Too many icons')
    
    print(N, W, H)
    
    sheet = Image.new('RGBA', (W*ICON_SIZE, H*ICON_SIZE))
    print(f'Created texture atlas {sheet.size}')
    
    icon_map = {}
    for i, icon_name in enumerate(icons):
        x = i % W
        y = i // W
        cell_index = y*W + x
        print('.', x, y, cell_index, icon_name)
        
        img = cc.get_icon(icon_name)
        with io.BytesIO(img) as buf:
            img = Image.open(buf)
            
            ISIZE = ICON_SIZE, ICON_SIZE
            if img.size>ISIZE:
                img = img.thumbnail(ISIZE)
            
            if img.size[0]<ICON_SIZE or img.size[1]<ICON_SIZE:
                img_size = Image.new('RGBA', ISIZE, (255, 255, 255, 0))
                dx = (ICON_SIZE-img.size[0])//2
                dy = (ICON_SIZE-img.size[1])//2
                img_size.paste(img, (dx, dy))
                img = img_size
        
            print('  ', (x*ICON_SIZE, y*ICON_SIZE))
            sheet.paste(img, (x*ICON_SIZE, y*ICON_SIZE))
            icon_map[icon_name] = y*N + i
    
    print('Save texture atlas to {atlas_name}')
    sheet.save(atlas_name)
    
    return W, H, icon_map

In [ ]:
icons = set(vxdf['icon']) | set(vxdf['background_icon'])
width, height, icon_map = make_texture_atlas(icons, SPRITE_ATLAS)
vxdf['fg_icon_index'] = vxdf.icon.apply(lambda icon_name:icon_map[icon_name] if icon_name else 0)
vxdf['bg_icon_index'] = vxdf.background_icon.apply(lambda icon_name:icon_map[icon_name] if icon_name else 0)

In [ ]:
vxs = {}
for vx in vxdf.to_dict(orient='records'):
    vx_id = vx.pop('[id]')
    print(vx)
    vxs[vx_id] = vx
vxs

## Transactions

In [ ]:
tx_attrs = ['source.[id]', 'destination.[id]', 'transaction.color']

txdf = cc.get_dataframe(tx=True, attrs=tx_attrs)
txdf = txdf.rename(columns=lambda name:name.replace('transaction.', '', 1) if name.startswith('transaction.') else name)

print(txdf.shape)
txdf.head()

In [ ]:
txdf = txdf.drop_duplicates(['source.[id]', 'destination.[id]'])
print(txdf.shape)

In [ ]:
txs = txdf.to_dict(orient='records')
txs

In [ ]:
with open('sphere-graph.json', 'w') as f:
    json.dump({
        'vertex': vxs,
        'transaction': txs,
        'sprite_atlas': {
            'name': SPRITE_ATLAS,
            'width': width,
            'height': height
            },
        'icon': icon_map
        }, f, indent=2)

In [ ]:
d = {'a':1, 'b':2}
d.pop('a')
d

In [ ]:
Image.new?

In [ ]:
(3,5)>(3,3)